In [6]:
import pandas as pd

path = '../csv_output/'
data_may = pd.read_csv(path+'BDVPerChannelMar21')
data_june = pd.read_csv(path+'BDVPerChannelMar21')

data_all = pd.concat([data_may, data_june])

FileNotFoundError: [Errno 2] No such file or directory: '../csv_output/BDVPerChannelMar21'

In [26]:
data_all['chd_direction'] = 0
data_all.loc[(data_all['qdv_main_drive_position'].diff() > 0) & (data_all['qdv_main_drive_position'].diff()< 100), 'chd_direction'] = 1
data_all.loc[(data_all['qdv_main_drive_position'].diff() < 0) & (data_all['qdv_main_drive_position'].diff()> -100), 'chd_direction'] = -1
#data_all = data_all.sort_values('count') #.dropna(subset='count')

In [27]:
from datetime import timedelta
data_cam = pd.read_csv(path+'TBM_data/cam/BoulderAnalysisNew.csv')
data_cam['Time (Eastern Standard Time)'] = pd.to_datetime(data_cam['Time (Eastern Standard Time)'])
data_cam['Time (Eastern Standard Time)'] = data_cam['Time (Eastern Standard Time)'] +timedelta(minutes=1, seconds=10)

#data_all = data_all.merge(data_cam, left_on='time', right_on='Time (Eastern Standard Time)', how='outer')

In [28]:
# when cam was off
cam_on = [[ '2023-05-23 15:00:00','2023-05-24 00:00:00'], 
          [ '2023-05-24 06:00:00','2023-05-25 00:00:00'], 
          [ '2023-05-25 08:00:00','2023-05-25 16:00:00'],   
          [ '2023-06-01 13:24:00','2023-06-02 00:00:00'], 
          [ '2023-06-02 00:00:00','2023-06-03 00:00:00'], 
          [ '2023-06-03 00:00:00','2023-06-04 00:00:00'], 
          [ '2023-06-04 00:00:00','2023-06-05 00:00:00'], 
          [ '2023-06-05 00:00:00','2023-06-06 00:00:00'], 
          [ '2023-06-06 00:00:00','2023-06-07 00:00:00'], 
        ]

In [29]:
import plotly.graph_objects as go; import numpy as np
from plotly.subplots import make_subplots
from plotly_resampler import FigureResampler, FigureWidgetResampler
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)  


row=7
def plot(data_advance):
    fig =  FigureResampler(make_subplots(rows = row, cols=1, vertical_spacing = 0.025,shared_xaxes=True,  
                                                                   specs=[[{"secondary_y": True}]]*row
                                                                       ))

    #fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='black'), text=data['QWV_Advance_number'].values, hovertemplate='adv#: %{text}',), hf_x=data['Time'], hf_y=data['QDV_Advance_speed'], col=1, row=1)

    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='black')), hf_x=data_advance['count'], hf_y=data_advance['qdv_advance_speed_fixed'], col=1, row=1)
    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='grey')), hf_x=data_advance['count'], hf_y=data_advance['qdv_main_drive_speed'], col=1, row=2)
    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, mode = 'markers', marker=dict(color='red')), hf_x=data_advance.loc[data_advance['chd_direction']>0, 'count'], hf_y=  data_advance.loc[data_advance['chd_direction']>0, 'qdv_main_drive_speed'], col=1, row=2)
    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, mode = 'markers', marker=dict(color='blue')), hf_x=data_advance.loc[data_advance['chd_direction']<0, 'count'], hf_y=  data_advance.loc[data_advance['chd_direction']<0, 'qdv_main_drive_speed'], col=1, row=2)
    
    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='grey')), hf_x=data_advance['count'], hf_y=data_advance['qdv_screw_conveyor_speed'], col=1, row=3)
    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='grey')), hf_x=data_advance['count'], hf_y=data_advance['qdv_back_up_belt_01_speed'], col=1, row=4)

    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='black')), hf_x=data_advance['count'], hf_y=data_advance['bdv_e'], col=1, row=5)
    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='green')), hf_x=data_advance['count'], hf_y=data_advance['bdv_b3z'], col=1, row=5)


    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='black')), hf_x=data_advance['count'], hf_y=data_advance['bdv_c'], col=1, row=6)
    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='green')), hf_x=data_advance['count'], hf_y=data_advance['bdv_d'], col=1, row=6)

    fig.add_trace(go.Scattergl(name='noisy sine', showlegend=False, line=dict(color='blue')), hf_x=data_advance['count'], hf_y=data_advance['bdv_a3z'], col=1, row=7)

    
    for time in data_cam.loc[data_cam[' Rock Size']==0.5, 'Time (Eastern Standard Time)'].values:
        fig.add_shape(type="line", yref="paper", 
            x0=time, y0=0, x1=time, y1=1,
            line=dict(color="orange",width=1, dash = 'dash')
        )
    for time in data_cam.loc[data_cam[' Rock Size']==1, 'Time (Eastern Standard Time)'].values:
        fig.add_shape(type="line", yref="paper", 
            x0=time, y0=0, x1=time, y1=1,
            line=dict(color="red",width=1, dash = 'dash')
        )
    for time in cam_on:
        fig.add_vrect(
        x0=time[0], x1=time[1],
        fillcolor="green", opacity=0.1,
        layer="below", line_width=0,
        )
    
    fig.update_yaxes(title='', showgrid=True,  gridwidth=0.5, gridcolor='lightgrey',tickfont=dict(size=19),
                               ticks='inside' ,   
                               linecolor= 'black',
                         zerolinecolor= 'black',
                                linewidth= 1,
                         zerolinewidth= 1,
                                mirror= True)

    fig.update_xaxes(title='',  showgrid=True,  gridwidth=0.5, gridcolor='lightgrey', tickfont=dict(size=19),
                               ticks='inside' ,   
                               linecolor= 'black',
                                linewidth= 1,
                                mirror= True,
                             #  dtick=5,
    #                                     tickmode = 'array',
    #                                    tickvals = np.array(indexes),
    #                                    ticktext = unique 
                    )

    fig.update_yaxes(title='AR, mm/min', range=[0,60], col=1, row=1)
    fig.update_yaxes(title='Nchd, rpm', range=[0,2], col=1, row=2)
    fig.update_yaxes(title='Nsc, rpm', range=[0,8], col=1, row=3)
    fig.update_yaxes(title='Belt speed', range=[0,200], col=1, row=4)

    fig.update_yaxes(title='BDV-(R)', range=[0,1], col=1, row=5)
    fig.update_yaxes(title='BDV-(L)', range=[0,1], col=1, row=6)
    fig.update_yaxes(title='BDV-(sc)', range=[0,1], col=1, row=7)




    fig.update_layout(title = '(Eastern Time)', paper_bgcolor='white', plot_bgcolor='white',
                              height=900, width=1320, #width=1800, 

                            font=dict(
                                family="Helvetica",
                                size=16,
                                color="black"
                            ),

                             )
    fig.show()
    #fig.show_dash(port='1001')
plot(data_all)